In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import time
import dill
import pandas as pd
import plotly.express as ply_exp
import plotly.graph_objects as ply_go
import plotly.figure_factory as fig_fac

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada

# gpus = tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
# tf.config.experimental.set_memory_growth(gpus[0], True)    # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [2]:
def Z_plus_mapping(ω, API, s):
    
    α = s*ω + ((1/100)**(2/3))*(API.abs(s*ω-1))**(1/2)
    
    return α

Z_2_mapping = lambda ω, API: Z_plus_mapping(ω, API, 10e2)
Z_3_mapping = lambda ω, API: Z_plus_mapping(ω, API, 10e3)
Z_4_mapping = lambda ω, API: Z_plus_mapping(ω, API, 10e4)
    

In [3]:
nomes   = ["Symmetric Mapping", "BI Mapping", "BIm Mapping", "IM(2,0.1) Mapping", "ENO Mapping", "BI2 Mapping"] 

mapping = {}
mapping[nomes[0]] = Hong_mapping
mapping[nomes[1]] = BI_mapping
mapping[nomes[2]] = BIm_mapping
mapping[nomes[3]] = IM_mapping 
mapping[nomes[4]] = ENO_mapping
mapping[nomes[5]] = BI2_mapping

API = API_Numpy


In [4]:
x, y = np.meshgrid(
    np.arange(0.0, 1.02, 0.02),
    np.arange(0.0, 1.02, 0.02)
)

for i in range(len(x)):
    for j in range(len(x[0])):
        if x[i][j] + y[i][j] > 1.01:
            x[i][j] = 0
            y[i][j] = 0

z  = 1-(x+y)
ω0 = np.concatenate([[x, z, y]], axis = 0)
ω0 = np.transpose(ω0, (1, 2, 0))

for nome in nomes:
    
    ω1 = mapping[nome](ω0, API)
    ω1 = ω1/np.sum(ω1, axis = -1, keepdims = True)
    
    u = ω1[...,0:1]-ω0[...,0:1]
    v = ω1[...,2: ]-ω0[...,2: ]
    
    u = np.squeeze(u)
    v = np.squeeze(v)
    
    fig = fig_fac.create_quiver(
        x, y, u, v,
        scale       = 0.1,
        arrow_scale = 0.2,
        name        = nome,
        line_width  = 1.5,
        line_color  = "#4262C1"
    )
    
    fig.update_layout(
        paper_bgcolor = "#FFFFFF",
        plot_bgcolor  = "#FFFFFF",
        xaxis         = dict(
            linecolor = "#000000"              , 
            ticks     = "inside"               ,
            title     = r"$\lambda_{0}$"       ,
            tickvals  = np.arange(0, 1.1, 0.1)
        ),
        yaxis         = dict(
            linecolor = "#000000"              ,
            ticks     = "inside"               ,
            title     = r"$\lambda_{2}$"       ,
            tickvals  = np.arange(0, 1.1, 0.1)
        ),
        legend = dict(
            title       = ""        ,
            yanchor     = "top"     ,
            y           = 0.99      ,
            xanchor     = "right"   ,
            x           = 0.99      ,
            bordercolor = "#666666" ,
            borderwidth = 1
        )
    )
    
    fig.add_trace(
        ply_go.Scatter(
            x    = np.concatenate([
                tf.range(0, 1.01, 0.01, dtype=dtype)  ,
                tf.range(0, 1.01, 0.01, dtype=dtype)*0,
                tf.range(0, 1.01, 0.01, dtype=dtype)
            ]),
            y    = np.concatenate([
                  tf.range(0, 1.01, 0.01, dtype=dtype)*0,
                  tf.range(0, 1.01, 0.01, dtype=dtype)  ,
                1-tf.range(0, 1.01, 0.01, dtype=dtype)
            ]),
            mode = "lines"                              ,
            line = ply_go.scatter.Line(color="#666666") ,
            name = ""                                   ,
            showlegend = False
        )
    )
    
    fig.add_trace(
        ply_go.Scatter(
            x = [-0.01]               ,
            y = [0.5]                ,
            mode = "markers"         ,
            name = ""                ,
            marker_size  = 1         ,
            marker_color = "#FFFFFF" ,
            showlegend = False
        )
    )
    
#     fig.add_trace(
#         ply_go.Scatter(
#             x = [1/3]                        ,
#             y = [1/3]                        ,
#             mode = "markers"                 ,
#             name = r"$5^{th} \text{ order}$" ,
#             marker_size  = 6                 ,
#             marker_color = "#4262C1"
#         )
#     )

#     fig.add_trace(
#         ply_go.Scatter(
#             x = [0  , 2/3]                   ,
#             y = [2/3, 0  ]                   ,
#             mode = "markers"                 ,
#             name = r"$4^{th} \text{ order}$" ,
#             marker_size  = 6                 ,
#             marker_color = "#C23C52"
#         )
#     )

#     fig.add_trace(
#         ply_go.Scatter(
#             x = [0  , 1/2, 1, 0, 0]          ,
#             y = [1/2, 0  , 0, 1, 0]          ,
#             mode = "markers"                 ,
#             name = r"$3^{rd} \text{ order}$" ,
#             marker_size  = 6                 ,
#             marker_color = "#57933B"
#         )
#     )

#     fig.show()
    fig.write_html("htmls/mapeamentos/" + nome + ".html")
    fig.write_image("htmls/mapeamentos/" + nome + ".png", width = 720, height = 720)
    print(nome)

Symmetric Mapping
BI Mapping
BIm Mapping
IM(2,0.1) Mapping
ENO Mapping
BI2 Mapping
